In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/22 16:49:02 WARN Utils: Your hostname, NOTEDELL3420 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/22 16:49:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/codigos/spark-iceberg/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jlsilva01/.ivy2/cache
The jars for the packages stored in: /home/jlsilva01/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3f60b2c7-2807-4ea1-8351-30226c7c047d;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.6.1/iceberg-spark-runtime-3.5_2.12-1.6.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1!iceberg-spark-runtime-3.5_2.12.jar (3098ms)
:: resolution report :: resolve 1150ms :: artifacts dl 3100ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [2]:
spark

In [16]:
spark.sql(
  """
  CREATE TABLE local.carro_iceberg (id INT, placa STRING) USING iceberg
  """
)

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `carro_iceberg` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [17]:
spark.sql("select * from local.carro_iceberg").show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|     GM|tracker|2020|
+---+-------+-------+-------+----+



In [18]:
spark.sql("INSERT INTO local.carro_iceberg VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

AnalysisException: [INSERT_COLUMN_ARITY_MISMATCH.NOT_ENOUGH_DATA_COLUMNS] Cannot write to `local`.`carro_iceberg`, the reason is not enough data columns:
Table columns: `id`, `placa`, `marca`, `modelo`, `ano`.
Data columns: `col1`, `col2`.

In [19]:
spark.sql("select * from local.carro_iceberg").show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|     GM|tracker|2020|
+---+-------+-------+-------+----+



In [20]:
spark.sql(
    """
    alter table local.carro_iceberg add column marca STRING, modelo STRING, ano INT
    """
)

AnalysisException: [FIELDS_ALREADY_EXISTS] Cannot add column, because `marca` already exists in "STRUCT<id: INT, placa: STRING, marca: STRING, modelo: STRING, ano: INT>".; line 2 pos 4;
AddColumns [QualifiedColType(None,marca,StringType,true,None,None,None), QualifiedColType(None,modelo,StringType,true,None,None,None), QualifiedColType(None,ano,IntegerType,true,None,None,None)]
+- ResolvedTable org.apache.iceberg.spark.SparkCatalog@48cc1799, carro_iceberg, local.carro_iceberg, [id#428, placa#429, marca#430, modelo#431, ano#432]


In [21]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+



In [22]:
spark.sql(
    """
    update local.carro_iceberg set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[]

In [23]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+



In [24]:
spark.sql('update local.carro_iceberg set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[]

In [25]:
spark.sql('update local.carro_iceberg set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[]

In [26]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+



In [27]:
spark.sql('delete from local.carro_iceberg where id = 3')

DataFrame[]

In [28]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+

